# [Report PDF Here](https://drive.google.com/file/d/1Tzsoo9VaZlKZGNfmsAWHPDsBBR7iVab0/view?usp=sharing)

In [ ]:
--STEP 1 DVC DEEP DIVE COUNTY EXPORT

(select attempts.report_time,
	'hold' as region_name,
    tot.county_name,
    vbm.vbm_commits,
    attempts.total_attempts,
    canv.total_canvassed,
 	calls.call_attempts,
 	pcan.calls_canvassed,
 texts.text_attempts,
 tcan.texts_canvassed,
 tot.targets,
 ppl.ppl_attempted,
 active.active_targets,
 active_attempted.active_ppl_attempted,
 vbm_stat.vbm_stat_count,
 ids.total_ids,
 dem_ids.total_dem_ids
from
 /* Total Targets */
(select convert_timezone('America/Chicago',getdate()) as report_time,
t2.county_name,
count(distinct(t1.myv_van_id)) as targets
from states_ia_universes.current_universe_reporting t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.turf t3 on t2.van_precinct_id = t3.van_precinct_id
group by 1, 2) tot
 /* Total Attempts */
left join 
(select
convert_timezone('America/Chicago',getdate()) as report_time,
 	t2.county_name,
    count(t1.myv_van_id) as total_attempts
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_phone = 1
         or (t3.attempt_text = 1 and t3.contact_type_name = 'SMS Text'))
group by 1, 2) attempts on tot.county_name = attempts.county_name
left join
 /* Total Canvassed*/
(select
 	t2.county_name,
   count(distinct(t3.myv_van_id)) as total_canvassed
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.phone_canvassed = 1
     or (t3.text_canvassed = 1 and t3.contact_type_name = 'SMS Text'))
group by 1) canv on attempts.county_name = canv.county_name
left join   
/* VBM Commits */
(select
 	t2.county_name,
    count(distinct t1.myv_van_id) as vbm_commits
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ
 	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
      or survey_response_id = 1660742 -- Y: Commit to Request 
      or survey_response_id = 1660745 -- Y: Commit to Return
      or survey_response_id = 1660748 -- 'Y: ABR Collected'
      or survey_response_id = 1660747) --'Y: Already Returned'
group by 1) vbm
on vbm.county_name = attempts.county_name
left join
 /* Call Attempts */
(select 
 	t2.county_name,
 	sum(t3.attempt_phone) as call_attempts
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.attempt_phone = '1'
group by 1) calls on attempts.county_name = calls.county_name
left join 
 /* Text Attempts */
(select 
 	t2.county_name,
 	sum(t3.attempt_text) as text_attempts
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_text = '1' and t3.contact_type_name = 'SMS Text')
group by 1) texts on attempts.county_name = texts.county_name
 /* Calls Canvassed */
left join
(select
 	t2.county_name,
 	count(distinct(t3.myv_van_id)) as calls_canvassed
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.contact_type_name = 'Phone'
	and t3.attempt_phone = 1
 	and t3.phone_canvassed = 1
group by 1) pcan on attempts.county_name = pcan.county_name
left join
 /* Text Canvassed */
(select 
 	t2.county_name,
 	count(distinct(t3.myv_van_id)) as texts_canvassed
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.text_canvassed = '1' and t3.contact_type_name = 'SMS Text')
group by 1) tcan on attempts.county_name = tcan.county_name
left join 
 /* People Attempted */
(select
convert_timezone('America/Chicago',getdate()) as report_time,
 	t2.county_name,
    count(distinct(t1.myv_van_id)) as ppl_attempted
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_phone = 1
         or t3.attempt_text = 1)
group by 1, 2) ppl on attempts.county_name = ppl.county_name
left join
/* Total Active Targets */
(select t3.county_name,
 	count(distinct(t1.myv_van_id)) as active_targets
from states_ia_universes.current_universe_reporting t1
left join
(select myv_van_id
 from my_state_van.targets_myv
 where target_subgroup_name = 'DO NOT CALL - Suppressions') t2 on t1.myv_van_id = t2.myv_van_id
left join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
where t2.myv_van_id is null
group by 1) active on attempts.county_name = active.county_name
left join
/* # of Attempted Active Targets */
(select t3.county_name,
	count(distinct(t1.myv_van_id)) as active_ppl_attempted
from states_ia_universes.current_universe_reporting t1
left join
(select myv_van_id
 from my_state_van.targets_myv
 where target_subgroup_name = 'DO NOT CALL - Suppressions') t2 on t1.myv_van_id = t2.myv_van_id
left join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
left join my_state_van.coord20_myv_001_contacts t4 on t1.myv_van_id = t4.myv_van_id
where t2.myv_van_id is null
	and (t4.attempt_phone = 1
         or (t4.attempt_text = 1 and t4.contact_type_name = 'SMS Text'))
group by 1) active_attempted on attempts.county_name = active_attempted.county_name
left join
/* VBM Status Count */
(select
 	t2.county_name,
    count(distinct(t1.myv_van_id)) as vbm_stat_count
  from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ 
  and t3.reporting_week_start_date >= '2020-07-31'
	and t3.reporting_week_start_date < current_date
group by 1) vbm_stat on tot.county_name = vbm_stat.county_name
left join
/* Total IDs Collected */
(select t2.county_name,
 	count(distinct(t3.myv_van_id)) as total_ids
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
	and t3.sq_most_recent_response = 1
group by 1) ids on tot.county_name = ids.county_name
left join
/* Total Dem IDs Collected */
(select t2.county_name,
 	count(distinct(t3.myv_van_id)) as total_dem_ids
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
	and t3.sq_most_recent_response = 1
 	and (survey_response_id = 1668845	-- 1 - Strong Democrat
         or survey_response_id = 1668846)	-- 2 - Lean Democrat
group by 1) dem_ids on tot.county_name = dem_ids.county_name);

--STEP 2
-- DVC Deep Dive Canvas Results by County Export
select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    attempts.county_name,
    attempts.reporting_week_start_date,
    attempts.total_attempts,
    ids.total_ids,
    (select count(distinct(myv_van_id))
      from states_ia_universes.current_universe_reporting) as total_universe,
    attempts_p2.total_attempts_p2,
    ids_p2.total_ids_p2,
    (select count(distinct(myv_van_id))
      from states_ia_universes.dvc_phase2_08292020) as p2_universe
from
/* Total Attempts All Phase*/
(select
 	t2.county_name,
  	t3.reporting_week_start_date,
    count(t1.myv_van_id) as total_attempts
from states_ia_universes.current_universe_reporting t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_phone = 1
         or (t3.attempt_text = 1 and contact_type_name = 'SMS Text'))
group by 1, 2) attempts
full join
/* Total IDs Collected All Phase */
(select
 	t2.county_name,
 	t3.reporting_week_start_date,
 	count(distinct(t3.myv_van_id)) as total_ids
from states_ia_universes.current_universe_reporting t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
	and t3.sq_most_recent_response = 1
group by 1, 2) ids on attempts.county_name = ids.county_name and attempts.reporting_week_start_date = ids.reporting_week_start_date
full join
/* Total Attempts 2nd Phase Phase*/
(select
 	t2.county_name,
  	t3.reporting_week_start_date,
    count(t1.myv_van_id) as total_attempts_p2
from states_ia_universes.dvc_phase2_08292020 t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_phone = 1
         or (t3.attempt_text = 1 and contact_type_name = 'SMS Text'))
group by 1, 2) attempts_p2 on attempts.county_name = attempts_p2.county_name and attempts.reporting_week_start_date = attempts_p2.reporting_week_start_date
full join
/* Total IDs Collected 2nd Phase */
(select
 	t2.county_name,
 	t3.reporting_week_start_date,
 	count(distinct(t3.myv_van_id)) as total_ids_p2
from states_ia_universes.dvc_phase2_08292020 t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
	and t3.sq_most_recent_response = 1
group by 1, 2) ids_p2 on attempts.county_name = ids_p2.county_name and attempts.reporting_week_start_date = ids_p2.reporting_week_start_date;

-- STEP 3
-- DVC Deep Dive Map Info for Carto Export
(select
    attempts.county_name,
 	'' as region_name,
    vbm.vbm_commits,
    attempts.total_attempts,
    canv.total_canvassed,
 	calls.call_attempts,
 	pcan.calls_canvassed,
 	texts.text_attempts,
 	tcan.texts_canvassed
from
(select
 	t2.county_name,
    count(t1.myv_van_id) as total_attempts
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_phone = 1
         or (t3.attempt_text = 1 and contact_type_name = 'SMS Text'))
group by 1) attempts 
left join
(select
 	t2.county_name,
   count(distinct(t3.myv_van_id)) as total_canvassed
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.phone_canvassed = 1
     or (t3.text_canvassed = 1 and contact_type_name = 'SMS Text'))
group by 1) canv on attempts.county_name = canv.county_name
left join                              
(select
 	t2.county_name,
    count(distinct t1.myv_van_id) as vbm_commits
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ
 	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
      or survey_response_id = 1660742 -- Y: Commit to Request 
      or survey_response_id = 1660745) -- Y: Commit to Return 
group by 1) vbm
on vbm.county_name = attempts.county_name
left join
(select
 	t2.county_name,
 	sum(t3.attempt_phone) as call_attempts
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.contact_type_name = 'Phone'
	and t3.attempt_phone = '1'
group by 1) calls on attempts.county_name = calls.county_name
left join 
(select 
 	t2.county_name,
 	sum(t3.attempt_text) as text_attempts
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_text = '1' and contact_type_name = 'SMS Text')
group by 1) texts on attempts.county_name = texts.county_name
left join
(select
 	t2.county_name,
 	count(distinct(t3.myv_van_id)) as calls_canvassed
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.contact_type_name = 'Phone'
	and t3.attempt_phone = 1
 	and t3.phone_canvassed = 1
group by 1) pcan on attempts.county_name = pcan.county_name
left join
(select
 	t2.county_name,
 	count(distinct(t3.myv_van_id)) as texts_canvassed
from states_ia_universes.current_universe_reporting t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.attempt_text = '1'
 	and t3.text_canvassed = '1'
 	and contact_type_name = 'SMS Text'
group by 1) tcan on attempts.county_name = tcan.county_name);

-- STEP 4
-- DVC Deep Dive All Phase VBM Party Count Roll Up

(select
 	case 
 		when vbm.survey_response_id = 1660750 then 'E-Day Vote In-Person'
 		when vbm.survey_response_id = 1660747 then 'Y: Already Returned'
        when vbm.survey_response_id = 1660740 then 'Y: Commit Send ABR'
        when vbm.survey_response_id = 1660742 then 'Y: Commit to Request'
        when vbm.survey_response_id = 1660753 then 'Early Vote In-Person'
        when vbm.survey_response_id = 1660745 then 'Y: Commit to Return'	
       	when vbm.survey_response_id = 1660748 then 'Y: ABR Collected'
        when vbm.survey_response_id = 1660754 then 'Not Voting'
 		when vbm.survey_response_id is null then 'VBM Status Unknown'
    end as metric,
 	count(distinct (dvc_univ.myv_van_id)) as metric_count
from states_ia_universes.current_universe_reporting dvc_univ 
left join (select myv_van_id,
           		  survey_response_id
				from my_state_van.coord20_myv_001_responses 
           		where survey_question_id = 406163 -- IA VBM Plan
					and sq_most_recent_response = 1) vbm on dvc_univ.myv_van_id = vbm.myv_van_id
group by 1)

union all

(select
 	case 
    	when ids.survey_response_id = 1668845 then 'Strong Democrat'
		when ids.survey_response_id = 1668846 then 'Lean Democrat'	
    	when ids.survey_response_id = 1668847 then 'Undecided'
    	when ids.survey_response_id = 1668848 then 'Lean Republican'
    	when ids.survey_response_id = 1668849 then 'Strong Republican'
 			when ids.survey_response_id is null then 'Not Canvassed'
    end as metric,
    count(distinct dvc_univ.myv_van_id) as metric_count
from states_ia_universes.current_universe_reporting dvc_univ 
left join (select myv_van_id,
           	survey_response_id
           from my_state_van.coord20_myv_001_responses
           where survey_question_id = 408235 -- ***IA Dem
           		and sq_most_recent_response = 1
           ) ids on dvc_univ.myv_van_id = ids.myv_van_id
 group by 1);
 
 -- STEP 5
 --DVC Deep Dive Party and Support and VBM Roll Up

(select 
 	case 
    	when ids.survey_response_id = 1668845 then 'Strong Democrat'
		when ids.survey_response_id = 1668846 then 'Lean Democrat'	
    	when ids.survey_response_id = 1668847 then 'Undecided'
    	when ids.survey_response_id = 1668848 then 'Lean Republican'
    	when ids.survey_response_id = 1668849 then 'Strong Republican'
 		when ids.survey_response_id is null then 'Not Canvassed'
    end as Party,
 	case
 		when bfp.bfp_support_score_targeting between 0.0 and 0.2 then 'BFP Support 0-20'
 		when bfp.bfp_support_score_targeting between 0.2 and 0.4 then 'BFP Support 20-40'
 		when bfp.bfp_support_score_targeting between 0.4 and 0.6 then 'BFP Support 40-60'
 		when bfp.bfp_support_score_targeting between 0.6 and 0.8 then 'BFP Support 60-80'
 		when bfp.bfp_support_score_targeting between 0.8 and 1.0 then 'BFP Support 80-100'
 		when bfp.bfp_support_score_targeting = '' then 'BFP Support Unknown'
 	end as metric,
 	count(distinct(bfp.person_id)) as metric_count
from states_ia_universes.current_universe_reporting dvc_univ 
left join (select myv_van_id,
           	survey_response_id
           from my_state_van.coord20_myv_001_responses
           where survey_question_id = 408235 -- ***IA Dem
           		and sq_most_recent_response = 1
           ) ids on dvc_univ.myv_van_id = ids.myv_van_id
left join 
 (select myv_van_id,
  		person_id
  from my_state.crosswalks_vanid_to_personid
  where state_code ='IA') cw on dvc_univ.myv_van_id = cw.myv_van_id
left join my_state.all_scores bfp on cw.person_id = bfp.person_id
group by 1, 2)

union all

(select 
 	case 
    	when ids.survey_response_id = 1668845 then 'Strong Democrat'
		when ids.survey_response_id = 1668846 then 'Lean Democrat'	
    	when ids.survey_response_id = 1668847 then 'Undecided'
    	when ids.survey_response_id = 1668848 then 'Lean Republican'
    	when ids.survey_response_id = 1668849 then 'Strong Republican'
 		when ids.survey_response_id is null then 'Not Canvassed'
    end as Party,
 	case
 		when dscc.support_score between 0.0 and 20.0 then 'Greenfield Support 0-20'
 		when dscc.support_score between 20.0 and 40.0 then 'Greenfield Support 20-40'
 		when dscc.support_score between 40.0 and 60.0 then 'Greenfield Support 40-60'
 		when dscc.support_score between 60.0 and 80.0 then 'Greenfield Support 60-80'
 		when dscc.support_score between 80.0 and 100.0 then 'Greenfield Support 80-100'
 		when dscc.support_score = '' then 'Greenfield Support Unknown'
 	end as metric,
 	count(distinct(dscc.person_id)) as metric_count
from states_ia_universes.current_universe_reporting dvc_univ 
left join (select myv_van_id,
           	survey_response_id
           from my_state_van.coord20_myv_001_responses
           where survey_question_id = 408235 -- ***IA Dem
           		and sq_most_recent_response = 1
           ) ids on dvc_univ.myv_van_id = ids.myv_van_id
left join 
 (select myv_van_id,
  		person_id
  from my_state.crosswalks_vanid_to_personid
  where state_code ='IA') cw on dvc_univ.myv_van_id = cw.myv_van_id
left join states_ia_projects.dscc_scores_august dscc on cw.person_id = dscc.person_id
group by 1, 2)

union all

(select 
 	case 
    	when ids.survey_response_id = 1668845 then 'Strong Democrat'
		when ids.survey_response_id = 1668846 then 'Lean Democrat'	
    	when ids.survey_response_id = 1668847 then 'Undecided'
    	when ids.survey_response_id = 1668848 then 'Lean Republican'
    	when ids.survey_response_id = 1668849 then 'Strong Republican'
 		when ids.survey_response_id is null then 'Not Canvassed'
    end as Party,
 	case 
 		when vbm.survey_response_id = 1660750 then 'E-Day Vote In-Person'
 		when vbm.survey_response_id = 1660747 then 'Y: Already Returned'
        when vbm.survey_response_id = 1660740 then 'Y: Commit Send ABR'
        when vbm.survey_response_id = 1660742 then 'Y: Commit to Request'
        when vbm.survey_response_id = 1660753 then 'Early Vote In-Person'
        when vbm.survey_response_id = 1660745 then 'Y: Commit to Return'	
       	when vbm.survey_response_id = 1660748 then 'Y: ABR Collected'
        when vbm.survey_response_id = 1660754 then 'Not Voting'
 		when vbm.survey_response_id is null then 'VBM Status Unknown'
    end as metric,
 count(distinct(vbm.myv_van_id))
from states_ia_universes.current_universe_reporting dvc_univ 
left join (select myv_van_id,
           	survey_response_id
           from my_state_van.coord20_myv_001_responses
           where survey_question_id = 408235 -- ***IA Dem
           		and sq_most_recent_response = 1
           ) ids on dvc_univ.myv_van_id = ids.myv_van_id
left join (select myv_van_id,
           		  survey_response_id
				from my_state_van.coord20_myv_001_responses 
           		where survey_question_id = 406163 -- IA VBM Plan
					and sq_most_recent_response = 1) vbm on dvc_univ.myv_van_id = vbm.myv_van_id
group by 1, 2
);

-- STEP 6
-- DVC Deep Dive VBM Status Support Roll Up
select 
 	case 
 		when vbm.survey_response_id = 1660750 then 'E-Day Vote In-Person'
 		when vbm.survey_response_id = 1660747 then 'Y: Already Returned'
        when vbm.survey_response_id = 1660740 then 'Y: Commit Send ABR'
        when vbm.survey_response_id = 1660742 then 'Y: Commit to Request'
        when vbm.survey_response_id = 1660753 then 'Early Vote In-Person'
        when vbm.survey_response_id = 1660745 then 'Y: Commit to Return'	
       	when vbm.survey_response_id = 1660748 then 'Y: ABR Collected'
        when vbm.survey_response_id = 1660754 then 'Not Voting'
 		when vbm.survey_response_id is null then 'VBM Status Unknown'
    end as VBM_Status, 
 	 	case
 		when bfp.bfp_support_score_targeting between 0.0 and 0.2 then 'BFP Support 0-20'
 		when bfp.bfp_support_score_targeting between 0.2 and 0.4 then 'BFP Support 20-40'
 		when bfp.bfp_support_score_targeting between 0.4 and 0.6 then 'BFP Support 40-60'
 		when bfp.bfp_support_score_targeting between 0.6 and 0.8 then 'BFP Support 60-80'
 		when bfp.bfp_support_score_targeting between 0.8 and 1.0 then 'BFP Support 80-100'
 		when bfp.bfp_support_score_targeting = '' then 'BFP Support Unknown'
 	end as metric,
 	count(distinct(bfp.person_id)) as metric_count
from states_ia_universes.current_universe_reporting dvc_univ 
left join (select myv_van_id,
           		  survey_response_id
				from my_state_van.coord20_myv_001_responses 
           		where survey_question_id = 406163 -- IA VBM Plan
					and sq_most_recent_response = 1) vbm on dvc_univ.myv_van_id = vbm.myv_van_id
left join 
 (select myv_van_id,
  		person_id
  from my_state.crosswalks_vanid_to_personid
  where state_code ='IA') cw on dvc_univ.myv_van_id = cw.myv_van_id
left join my_state.all_scores bfp on cw.person_id = bfp.person_id
group by 1, 2;

-- STEP 7
-- DVC Deep Dive Extended Universe

/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_distributed_09014 t1)

union all

/* Total Attempts */
(select 'total_attempts' as metric,
	count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_distributed_09014 t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total People Attempted */
(select 'ppl_attempted' as metric,
	count(distinct (t2.myv_van_id)) as metric_count
from states_ia_universes.tier4_persuasion_distributed_09014 t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total Canvassed */
(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.tier4_persuasion_distributed_09014 t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))

union all

/* Biden ID */

(select  
	case 
    	when t2.survey_response_id = 1658786 then 'POTUS- Strong Biden'
        when t2.survey_response_id = 1658787 then 'POTUS- Lean Biden'
        when t2.survey_response_id = 1658788 then 'POTUS- Undecided'
        when t2.survey_response_id = 1658789 then 'POTUS- Lean Trump'
        when t2.survey_response_id = 1658790 then 'POTUS- Strong Trump'
        when t2.survey_response_id = 1658791 then 'POTUS- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_distributed_09014 t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405714 --***IA POTUS ID
	and (t2.survey_response_id = 1658786	-- Strong Biden
         or t2.survey_response_id = 1658787	-- Lean Biden
         or t2.survey_response_id = 1658788	-- Undecided
         or t2.survey_response_id = 1658789	-- Lean Trump
         or t2.survey_response_id = 1658790	-- Strong Trump
         or t2.survey_response_id = 1658791)	-- Support Other
group by 1)

union all

/* Greenfield ID */
(select 
	case 
    	when t2.survey_response_id = 1658801 then 'Senate- Strong Greenfield'
        when t2.survey_response_id = 1658804 then 'Senate- Lean Greenfield'
        when t2.survey_response_id = 1658805 then 'Senate- Undecided'
        when t2.survey_response_id = 1658807 then 'Senate- Lean Ernst'
        when t2.survey_response_id = 1658810 then 'Senate- Strong Ernst'
        when t2.survey_response_id = 1658813 then 'Senate- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_distributed_09014 t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405716 --***IA US Senate ID
	and (t2.survey_response_id = 1658801	-- Strong Greenfiel
         or t2.survey_response_id = 1658804	-- Lean Greenfield
         or t2.survey_response_id = 1658805	-- Undecided
         or t2.survey_response_id = 1658807	-- Lean Ernst
         or t2.survey_response_id = 1658810	-- Strong Ernst
         or t2.survey_response_id = 1658813)	-- Support Other
group by 1)

union all

/* Dem ID */
(select 
	case 
    	when t2.survey_response_id = 1668845 then 'Party- Strong Democrat'
        when t2.survey_response_id = 1668846 then 'Party- Lean Democrat'
        when t2.survey_response_id = 1668847 then 'Party- Undecided'
        when t2.survey_response_id = 1668848 then 'Party- Lean Republican'
        when t2.survey_response_id = 1668849 then 'Party- Strong Republican'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_distributed_09014 t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 --***IA Dem ID
	and (survey_response_id = 1668845	-- Strong Democrat
         or survey_response_id = 1668846 -- Lean Democrat
         or survey_response_id = 1668847 -- Undecided
         or survey_response_id = 1668848 -- Lean Republican
         or survey_response_id = 1668849) -- Strong Republican
group by 1);

-- STEP 8
-- DVC Deep Dive All Voter Attempt Contact Method

select t1.contact_type,
	t1.attempt_count,
    t2.canvass_count
from
(select 
	case
    	when contact_type_name = 'No Actual Contact' then 'No Actual Contact'
        when contact_type_name = 'SMS Text' then 'SMS Text'
        when contact_type_name = 'Phone' then 'Phone'
        when contact_type_name = 'Distributed Text' then 'Distributed Text'
        when contact_type_name = 'Distributed Call' then 'Distributed Call'
        when contact_type_name = 'Auto Dial' then 'Auto Dial'
        when contact_type_name = 'Relational Text' then 'Relational Text'
        when contact_type_name = 'Paid Call' then 'Paid Call'
        else 'Other'
     end as contact_type,
	count(myv_van_id) as attempt_count
from my_state_van.coord20_myv_001_contacts
group by 1) t1
left join
(select 
	case
    	when contact_type_name = 'No Actual Contact' then 'No Actual Contact'
        when contact_type_name = 'SMS Text' then 'SMS Text'
        when contact_type_name = 'Phone' then 'Phone'
        when contact_type_name = 'Distributed Text' then 'Distributed Text'
        when contact_type_name = 'Distributed Call' then 'Distributed Call'
        when contact_type_name = 'Auto Dial' then 'Auto Dial'
        when contact_type_name = 'Relational Text' then 'Relational Text'
        when contact_type_name = 'Paid Call' then 'Paid Call'
        else 'Other'
     end as contact_type,
	count(myv_van_id) as canvass_count
from my_state_van.coord20_myv_001_contacts
where result_short_name = 'Canvassed'
group by 1) t2 on t1.contact_type = t2.contact_type;

-- STEP 9
-- DVC Deep Dive Tier 4 Metr

/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_limited t1)

union all

/* Total Attempts */
(select 'total_attempts' as metric,
	count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_limited t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total People Attempted */
(select 'ppl_attempted' as metric,
	count(distinct (t2.myv_van_id)) as metric_count
from states_ia_universes.tier4_persuasion_limited t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total Canvassed */
(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.tier4_persuasion_limited t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))

union all

/* Biden ID */

(select  
	case 
    	when t2.survey_response_id = 1658786 then 'POTUS- Strong Biden'
        when t2.survey_response_id = 1658787 then 'POTUS- Lean Biden'
        when t2.survey_response_id = 1658788 then 'POTUS- Undecided'
        when t2.survey_response_id = 1658789 then 'POTUS- Lean Trump'
        when t2.survey_response_id = 1658790 then 'POTUS- Strong Trump'
        when t2.survey_response_id = 1658791 then 'POTUS- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_limited t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405714 --***IA POTUS ID
	and (t2.survey_response_id = 1658786	-- Strong Biden
         or t2.survey_response_id = 1658787	-- Lean Biden
         or t2.survey_response_id = 1658788	-- Undecided
         or t2.survey_response_id = 1658789	-- Lean Trump
         or t2.survey_response_id = 1658790	-- Strong Trump
         or t2.survey_response_id = 1658791)	-- Support Other
group by 1)

union all

/* Greenfield ID */
(select 
	case 
    	when t2.survey_response_id = 1658801 then 'Senate- Strong Greenfield'
        when t2.survey_response_id = 1658804 then 'Senate- Lean Greenfield'
        when t2.survey_response_id = 1658805 then 'Senate- Undecided'
        when t2.survey_response_id = 1658807 then 'Senate- Lean Ernst'
        when t2.survey_response_id = 1658810 then 'Senate- Strong Ernst'
        when t2.survey_response_id = 1658813 then 'Senate- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_limited t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405716 --***IA US Senate ID
	and (t2.survey_response_id = 1658801	-- Strong Greenfiel
         or t2.survey_response_id = 1658804	-- Lean Greenfield
         or t2.survey_response_id = 1658805	-- Undecided
         or t2.survey_response_id = 1658807	-- Lean Ernst
         or t2.survey_response_id = 1658810	-- Strong Ernst
         or t2.survey_response_id = 1658813)	-- Support Other
group by 1)

union all

/* Dem ID */
(select 
	case 
    	when t2.survey_response_id = 1668845 then 'Party- Strong Democrat'
        when t2.survey_response_id = 1668846 then 'Party- Lean Democrat'
        when t2.survey_response_id = 1668847 then 'Party- Undecided'
        when t2.survey_response_id = 1668848 then 'Party- Lean Republican'
        when t2.survey_response_id = 1668849 then 'Party- Strong Republican'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier4_persuasion_limited t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 --***IA Dem ID
	and (survey_response_id = 1668845	-- Strong Democrat
         or survey_response_id = 1668846 -- Lean Democrat
         or survey_response_id = 1668847 -- Undecided
         or survey_response_id = 1668848 -- Lean Republican
         or survey_response_id = 1668849) -- Strong Republican
group by 1);

-- STEP 10
-- DVC Deep Dive Ballot Chase Universe

(select attempts.report_time,
	attempts.region_name,
    attempts.county_name,
    vbm.vbm_commits,
    attempts.total_attempts,
    canv.total_canvassed,
 	calls.call_attempts,
 	pcan.calls_canvassed,
 texts.text_attempts,
 tcan.texts_canvassed,
 tot.targets,
 ppl.ppl_attempted,
 vbm_stat.vbm_stat_count,
 ids.id_count,
 dem_ids.dem_id_count
from
 /* Total Targets */
(select convert_timezone('America/Chicago',getdate()) as report_time,
 	case
 		when t2.county_name = 'Jasper' then '7'
 		else t3.region_name
 	end as region_name,
t2.county_name,
count(distinct(t1.myv_van_id)) as targets
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.turf t3 on t2.van_precinct_id = t3.van_precinct_id
group by 1, 2, 3) tot
full join 
 /* Total Attempts*/
(select
convert_timezone('America/Chicago',getdate()) as report_time,
  case
 		when t2.county_name = 'Jasper' then '7'
 		else t3.region_name
 	end as region_name,
 	t2.county_name,
    count(t1.myv_van_id) as total_attempts
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_phone = 1
         or (t3.attempt_text = 1 and t3.contact_type_name = 'SMS Text'))
group by 1, 2, 3) attempts on tot.county_name = attempts.county_name and tot.region_name = attempts.region_name
full join
 /* Total Canvassed */
(select
 	t2.county_name,
   count(distinct(t3.myv_van_id)) as total_canvassed
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.phone_canvassed = 1
     or (t3.text_canvassed = 1 and t3.contact_type_name = 'SMS Text'))
group by 1) canv on attempts.county_name = canv.county_name
full join 
/* Total VBM Commits */
(select
 	t2.county_name,
    count(distinct t1.myv_van_id) as vbm_commits
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ
 	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
      or survey_response_id = 1660742 -- Y: Commit to Request 
      or survey_response_id = 1660745 -- Y: Commit to Return 
       or survey_response_id = 1660747 --'Y: Already Returned'
       or survey_response_id = 1660748) -- 'Y: ABR Collected'
group by 1) vbm
on vbm.county_name = attempts.county_name
full join
 /* Phone Attempts */
(select
 	t2.county_name,
 	sum(t3.attempt_phone) as call_attempts
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.attempt_phone = '1'
group by 1) calls on attempts.county_name = calls.county_name
full join 
 /* Text Attempts */
(select 
 	t2.county_name,
 	sum(t3.attempt_text) as text_attempts
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.attempt_text = '1'
  and t3.contact_type_name = 'SMS Text'
group by 1) texts on attempts.county_name = texts.county_name
full join
 /* Calls Canvassed */
(select 
 	t2.county_name,
 	count(distinct(t3.myv_van_id)) as calls_canvassed
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.phone_canvassed = 1
group by 1) pcan on attempts.county_name = pcan.county_name
full join
/* Text Canvassed */
(select 
 	t2.county_name,
 	count(distinct(t3.myv_van_id)) as texts_canvassed
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where t3.text_canvassed = '1'
  and t3.contact_type_name = 'SMS Text'
group by 1) tcan on attempts.county_name = tcan.county_name
full join
 /* People Attempted */
(select
 	t2.county_name,
    count(distinct(t1.myv_van_id)) as ppl_attempted
from states_ia_universes.ballot_chase_base_crosstabs t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.attempt_phone = 1
         or (t3.attempt_text = 1 and t3.contact_type_name = 'SMS Text'))
group by 1) ppl on attempts.county_name = ppl.county_name
left join
 /* VBM Status Count */
(select
 	t2.county_name,
    count(distinct(t1.myv_van_id)) as vbm_stat_count
  from states_ia_universes.ballot_chase_base_crosstabs t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ 
    and t3.reporting_week_start_date >= '2020-07-31'
	and t3.reporting_week_start_date < current_date
group by 1) vbm_stat on attempts.county_name = vbm_stat.county_name
left join
 /* ID Count */
(select
 	t2.county_name,
    count(distinct(t1.myv_van_id)) as id_count
  from states_ia_universes.ballot_chase_base_crosstabs t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 408235 -- ***IA Dem
    and t3.reporting_week_start_date >= '2020-07-31'
	and t3.reporting_week_start_date < current_date
group by 1) ids on attempts.county_name = ids.county_name
left join 
/* Dem IDs */
 (select
 	t2.county_name,
    count(distinct(t1.myv_van_id)) as dem_id_count
  from states_ia_universes.ballot_chase_base_crosstabs t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 408235 -- ***IA Dem
   	and (survey_response_id = 1668845	-- 1 - Strong Democrat
         or survey_response_id = 1668846)	-- 2 - Lean Democrat
    and t3.reporting_week_start_date >= '2020-07-31'
	and t3.reporting_week_start_date < current_date
group by 1) dem_ids on attempts.county_name = dem_ids.county_name);

-- STEP 11
-- Ballot Chase Roll Up
/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.ballot_chase_base t1)

union all

/* Total Attempts */
(select 'total_attempts' as metric,
	count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total People Attempted */
(select 'ppl_attempted' as metric,
	count(distinct (t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total Canvassed */
(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))

union all

/* Biden ID */

(select  
	case 
    	when t2.survey_response_id = 1658786 then 'POTUS- Strong Biden'
        when t2.survey_response_id = 1658787 then 'POTUS- Lean Biden'
        when t2.survey_response_id = 1658788 then 'POTUS- Undecided'
        when t2.survey_response_id = 1658789 then 'POTUS- Lean Trump'
        when t2.survey_response_id = 1658790 then 'POTUS- Strong Trump'
        when t2.survey_response_id = 1658791 then 'POTUS- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405714 --***IA POTUS ID
 	and t2.sq_most_recent_response = 1
	and (t2.survey_response_id = 1658786	-- Strong Biden
         or t2.survey_response_id = 1658787	-- Lean Biden
         or t2.survey_response_id = 1658788	-- Undecided
         or t2.survey_response_id = 1658789	-- Lean Trump
         or t2.survey_response_id = 1658790	-- Strong Trump
         or t2.survey_response_id = 1658791)	-- Support Other
group by 1)

union all

/* Greenfield ID */
(select 
	case 
    	when t2.survey_response_id = 1658801 then 'Senate- Strong Greenfield'
        when t2.survey_response_id = 1658804 then 'Senate- Lean Greenfield'
        when t2.survey_response_id = 1658805 then 'Senate- Undecided'
        when t2.survey_response_id = 1658807 then 'Senate- Lean Ernst'
        when t2.survey_response_id = 1658810 then 'Senate- Strong Ernst'
        when t2.survey_response_id = 1658813 then 'Senate- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405716 --***IA US Senate ID
 and t2.sq_most_recent_response = 1
	and (t2.survey_response_id = 1658801	-- Strong Greenfiel
         or t2.survey_response_id = 1658804	-- Lean Greenfield
         or t2.survey_response_id = 1658805	-- Undecided
         or t2.survey_response_id = 1658807	-- Lean Ernst
         or t2.survey_response_id = 1658810	-- Strong Ernst
         or t2.survey_response_id = 1658813)	-- Support Other
group by 1)

union all

/* Dem ID */
(select 
	case 
    	when t2.survey_response_id = 1668845 then 'Party- Strong Democrat'
        when t2.survey_response_id = 1668846 then 'Party- Lean Democrat'
        when t2.survey_response_id = 1668847 then 'Party- Undecided'
        when t2.survey_response_id = 1668848 then 'Party- Lean Republican'
        when t2.survey_response_id = 1668849 then 'Party- Strong Republican'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 --***IA Dem ID
 	and sq_most_recent_response = 1
	and (survey_response_id = 1668845	-- Strong Democrat
         or survey_response_id = 1668846 -- Lean Democrat
         or survey_response_id = 1668847 -- Undecided
         or survey_response_id = 1668848 -- Lean Republican
         or survey_response_id = 1668849) -- Strong Republican
group by 1)

union all

/* Ballot Chase SQ */

(select 
	case 
    	when t2.survey_response_id = 1736473 then 'Y: Already Mailed'
        when t2.survey_response_id = 1744703 then 'Y: Ballot Collected'
        when t2.survey_response_id = 1736481 then 'Y: Commit to Return'
 		when t2.survey_response_id = 1756210 then 'Y: Ballot PickUp Req'
 		when t2.survey_response_id = 1748753 then 'N: Not Voting'
        when t2.survey_response_id = 1736483 then 'N: Did not recieve'
        when t2.survey_response_id = 1736937 then 'N: Voting in Person'
    end as metric,
    count(t1.myv_van_id)
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 425204  -- IA Ballot Chase
	and (t2.survey_response_id = 1736473 -- Y: Already Mailed
         or t2.survey_response_id = 1744703 -- Y: Ballot Collected
         or t2.survey_response_id = 1736481	-- Y: Commit to Return
         or t2.survey_response_id = 1756210	-- Y: Ballot PickUp Req
         or t2.survey_response_id = 1748753	-- N: Not Voting
         or t2.survey_response_id = 1736483 -- N: Did not recieve
         or t2.survey_response_id = 1736937) -- N: Voting in Person
   and t2.sq_most_recent_response = 1
group by 1
order by 1)

union all

/* Phone Attempts */

(select
 	'Phone Attempts' as metric,
 	sum(t2.attempt_phone) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.attempt_phone = '1') 

union all

/* Text Attempts */

(select
 	'Text Attempts' as metric,
 	sum(t2.attempt_text) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.attempt_text = '1' and t2.contact_type_name = 'SMS Text') 

union all

/* Calls Canvassed */

(select
 	'Calls Canvassed' as metric,
 	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.phone_canvassed = 1) 

union all

/* Texts Canvassed */

(select
 	'Texts Canvassed' as metric,
 	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.text_canvassed = '1'
  and t2.contact_type_name = 'SMS Text');
  
-- STEP 12
--DVC Deep Dive Early Vote Roll Up

/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.evip_base t1)

union all

/* Total Attempts */
(select 'total_attempts' as metric,
	count(t2.myv_van_id) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total People Attempted */
(select 'ppl_attempted' as metric,
	count(distinct (t2.myv_van_id)) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total Canvassed */
(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))

union all

/* Biden ID */

(select  
	case 
    	when t2.survey_response_id = 1658786 then 'POTUS- Strong Biden'
        when t2.survey_response_id = 1658787 then 'POTUS- Lean Biden'
        when t2.survey_response_id = 1658788 then 'POTUS- Undecided'
        when t2.survey_response_id = 1658789 then 'POTUS- Lean Trump'
        when t2.survey_response_id = 1658790 then 'POTUS- Strong Trump'
        when t2.survey_response_id = 1658791 then 'POTUS- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405714 --***IA POTUS ID
  	and t2.sq_most_recent_response = 1
	and (t2.survey_response_id = 1658786	-- Strong Biden
         or t2.survey_response_id = 1658787	-- Lean Biden
         or t2.survey_response_id = 1658788	-- Undecided
         or t2.survey_response_id = 1658789	-- Lean Trump
         or t2.survey_response_id = 1658790	-- Strong Trump
         or t2.survey_response_id = 1658791)	-- Support Other
group by 1)

union all

/* Greenfield ID */
(select 
	case 
    	when t2.survey_response_id = 1658801 then 'Senate- Strong Greenfield'
        when t2.survey_response_id = 1658804 then 'Senate- Lean Greenfield'
        when t2.survey_response_id = 1658805 then 'Senate- Undecided'
        when t2.survey_response_id = 1658807 then 'Senate- Lean Ernst'
        when t2.survey_response_id = 1658810 then 'Senate- Strong Ernst'
        when t2.survey_response_id = 1658813 then 'Senate- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405716 --***IA US Senate ID
  	and t2.sq_most_recent_response = 1
	and (t2.survey_response_id = 1658801	-- Strong Greenfiel
         or t2.survey_response_id = 1658804	-- Lean Greenfield
         or t2.survey_response_id = 1658805	-- Undecided
         or t2.survey_response_id = 1658807	-- Lean Ernst
         or t2.survey_response_id = 1658810	-- Strong Ernst
         or t2.survey_response_id = 1658813)	-- Support Other
group by 1)

union all

/* Dem ID */
(select 
	case 
    	when t2.survey_response_id = 1668845 then 'Party- Strong Democrat'
        when t2.survey_response_id = 1668846 then 'Party- Lean Democrat'
        when t2.survey_response_id = 1668847 then 'Party- Undecided'
        when t2.survey_response_id = 1668848 then 'Party- Lean Republican'
        when t2.survey_response_id = 1668849 then 'Party- Strong Republican'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 --***IA Dem ID
  	and t2.sq_most_recent_response = 1
	and (survey_response_id = 1668845	-- Strong Democrat
         or survey_response_id = 1668846 -- Lean Democrat
         or survey_response_id = 1668847 -- Undecided
         or survey_response_id = 1668848 -- Lean Republican
         or survey_response_id = 1668849) -- Strong Republican
group by 1)

union all

/* Ballot Chase SQ */

(select 
	case 
    	when t2.survey_response_id = 1736473 then 'Y: Already Mailed'
        when t2.survey_response_id = 1744703 then 'Y: Ballot Collected'
        when t2.survey_response_id = 1736481 then 'Y: Commit to Return'
  		  when t2.survey_response_id = 1756210 then 'Y: Ballot PickUp Req'
 		    when t2.survey_response_id = 1748753 then 'N: Not Voting'
        when t2.survey_response_id = 1736483 then 'N: Did not recieve'
        when t2.survey_response_id = 1736937 then 'N: Voting in Person'
    end as metric,
    count(t1.myv_van_id) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 425204  -- IA Ballot Chase
  	and t2.sq_most_recent_response = 1
	and (t2.survey_response_id = 1736473 -- Y: Already Mailed
         or t2.survey_response_id = 1744703 -- Y: Ballot Collected
         or t2.survey_response_id = 1736481	-- Y: Commit to Return
         or t2.survey_response_id = 1756210	-- Y: Ballot PickUp Req
         or t2.survey_response_id = 1748753	-- N: Not Voting
         or t2.survey_response_id = 1736483 -- N: Did not recieve
         or t2.survey_response_id = 1736937) -- N: Voting in Person
   and t2.sq_most_recent_response = 1
group by 1
order by 1)

union all

/* Phone Attempts */

(select
 	'Phone Attempts' as metric,
 	sum(t2.attempt_phone) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.attempt_phone = '1') 

union all

/* Text Attempts */

(select
 	'Text Attempts' as metric,
 	sum(t2.attempt_text) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.attempt_text = '1' and t2.contact_type_name = 'SMS Text') 

union all

/* Calls Canvassed */

(select
 	'Calls Canvassed' as metric,
 	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.phone_canvassed = 1) 

union all

/* Texts Canvassed */

(select
 	'Texts Canvassed' as metric,
 	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.evip_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.text_canvassed = '1'
  and t2.contact_type_name = 'SMS Text');
  
-- STEP 13
--DVC Deep Dive Tier -1 IDS
/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.tier_neg1_for_dvc t1)

union all

/* Total Attempts */
(select 'total_attempts' as metric,
	count(t2.myv_van_id) as metric_count
from states_ia_universes.tier_neg1_for_dvc t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total People Attempted */
(select 'ppl_attempted' as metric,
	count(distinct (t2.myv_van_id)) as metric_count
from states_ia_universes.tier_neg1_for_dvc t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total Canvassed */
(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.tier_neg1_for_dvc t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))

union all

/* Biden ID */

(select  
	case 
    	when t2.survey_response_id = 1658786 then 'POTUS- Strong Biden'
        when t2.survey_response_id = 1658787 then 'POTUS- Lean Biden'
        when t2.survey_response_id = 1658788 then 'POTUS- Undecided'
        when t2.survey_response_id = 1658789 then 'POTUS- Lean Trump'
        when t2.survey_response_id = 1658790 then 'POTUS- Strong Trump'
        when t2.survey_response_id = 1658791 then 'POTUS- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier_neg1_for_dvc t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405714 --***IA POTUS ID
 and sq_most_recent_response = 1
	and (t2.survey_response_id = 1658786	-- Strong Biden
         or t2.survey_response_id = 1658787	-- Lean Biden
         or t2.survey_response_id = 1658788	-- Undecided
         or t2.survey_response_id = 1658789	-- Lean Trump
         or t2.survey_response_id = 1658790	-- Strong Trump
         or t2.survey_response_id = 1658791)	-- Support Other
group by 1)

union all

/* Greenfield ID */
(select 
	case 
    	when t2.survey_response_id = 1658801 then 'Senate- Strong Greenfield'
        when t2.survey_response_id = 1658804 then 'Senate- Lean Greenfield'
        when t2.survey_response_id = 1658805 then 'Senate- Undecided'
        when t2.survey_response_id = 1658807 then 'Senate- Lean Ernst'
        when t2.survey_response_id = 1658810 then 'Senate- Strong Ernst'
        when t2.survey_response_id = 1658813 then 'Senate- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier_neg1_for_dvc t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405716 --***IA US Senate ID
 and sq_most_recent_response = 1
	and (t2.survey_response_id = 1658801	-- Strong Greenfiel
         or t2.survey_response_id = 1658804	-- Lean Greenfield
         or t2.survey_response_id = 1658805	-- Undecided
         or t2.survey_response_id = 1658807	-- Lean Ernst
         or t2.survey_response_id = 1658810	-- Strong Ernst
         or t2.survey_response_id = 1658813)	-- Support Other
group by 1)

union all

/* Dem ID */
(select 
	case 
    	when t2.survey_response_id = 1668845 then 'Party- Strong Democrat'
        when t2.survey_response_id = 1668846 then 'Party- Lean Democrat'
        when t2.survey_response_id = 1668847 then 'Party- Undecided'
        when t2.survey_response_id = 1668848 then 'Party- Lean Republican'
        when t2.survey_response_id = 1668849 then 'Party- Strong Republican'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.tier_neg1_for_dvc t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 --***IA Dem ID
 and sq_most_recent_response = 1
	and (survey_response_id = 1668845	-- Strong Democrat
         or survey_response_id = 1668846 -- Lean Democrat
         or survey_response_id = 1668847 -- Undecided
         or survey_response_id = 1668848 -- Lean Republican
         or survey_response_id = 1668849) -- Strong Republican
group by 1);

-- STEP 14
-- DVC Deep Dive Latino & County Roll Up

(select t2.county_name,
	count(distinct(t1.myv_van_id)) as total,
 	count(distinct(t3.myv_van_id)) as dvc_total,
 	count(distinct(t4.myv_van_id)) as early_vote_total,
 	count(distinct(t5.myv_van_id)) as ballot_chase_total,
 	count(t6.call_attempts) as call_attempts,
 	count(t7.text_attempts) as text_attempts,
 	count(distinct(t8.myv_van_id)) as total_ppl_attempted,
 	count(distinct(t9.myv_van_id)) as calls_canvassed,
 	count(distinct(t10.myv_van_id)) as text_canvassed,
 	count(distinct(t11.myv_van_id)) as total_canvassed
from
((select distinct(myv_van_id)
 from my_state.person
 where ethnicity_modeled = 'H')
 union
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_activist_codes
 where activist_code_id = 4752784)) t1
join my_state.person t2 on t1.myv_van_id=t2.myv_van_id
left join states_ia_universes.dvc_phase2_08292020 t3 on t1.myv_van_id = t3.myv_van_id
left join states_ia_universes.evip_base t4 on t1.myv_van_id = t4.myv_van_id 
left join states_ia_universes.ballot_chase_base t5 on t1.myv_van_id = t5.myv_van_id 
left join
/* Call Attempts */
(select myv_van_id,
 	sum(attempt_phone) as call_attempts
 from my_state_van.coord20_myv_001_contacts
 where attempt_phone = '1'
group by 1) t6 on t1.myv_van_id = t6.myv_van_id
left join
/* Text Attempts */
(select myv_van_id,
 	sum(attempt_text) as text_attempts
 from my_state_van.coord20_myv_001_contacts
 where (attempt_text = '1' and contact_type_name = 'SMS Text')
 group by 1) t7 on t1.myv_van_id = t7.myv_van_id
left join
/* Total People Attempted */
(select myv_van_id
 from my_state_van.coord20_myv_001_contacts
 where (attempt_text = '1' and contact_type_name = 'SMS Text')
	or (attempt_phone = '1')) t8 on t1.myv_van_id = t8.myv_van_id 
left join
/* Calls Canvassed */
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_contacts
 where contact_type_name = 'Phone'
	and attempt_phone = 1
 	and phone_canvassed = 1) t9 on t1.myv_van_id = t9.myv_van_id 
left join
/* Texts Canvassed */
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_contacts
 where text_canvassed = '1' and contact_type_name = 'SMS Text') t10 on t1.myv_van_id = t10.myv_van_id
left join
/* Total Canvassed */
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_contacts
 where ((text_canvassed = '1' and contact_type_name = 'SMS Text') or phone_canvassed = 1)) t11 on t1.myv_van_id = t11.myv_van_id
group by 1
order by 1);

-- STEP 15
-- DVC Deep Dive Latino IDs
/* Biden ID */

(select  
 	c.county_name,
	case 
    	when t2.survey_response_id = 1658786 then 'POTUS- Strong Biden'
        when t2.survey_response_id = 1658787 then 'POTUS- Lean Biden'
        when t2.survey_response_id = 1658788 then 'POTUS- Undecided'
        when t2.survey_response_id = 1658789 then 'POTUS- Lean Trump'
        when t2.survey_response_id = 1658790 then 'POTUS- Strong Trump'
        when t2.survey_response_id = 1658791 then 'POTUS- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from my_state.person c
join
((select distinct(myv_van_id)
 from my_state.person
 where ethnicity_modeled = 'H')
 union
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_activist_codes
 where activist_code_id = 4752784)) t1 on c.myv_van_id = t1.myv_van_id
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405714 --***IA POTUS ID
	and (t2.survey_response_id = 1658786	-- Strong Biden
         or t2.survey_response_id = 1658787	-- Lean Biden
         or t2.survey_response_id = 1658788	-- Undecided
         or t2.survey_response_id = 1658789	-- Lean Trump
         or t2.survey_response_id = 1658790	-- Strong Trump
         or t2.survey_response_id = 1658791)	-- Support Other
 	and t2.sq_most_recent_response = 1
group by 1, 2)

union all

/* Greenfield ID */

(select c.county_name, 
	case 
    	when t2.survey_response_id = 1658801 then 'Senate- Strong Greenfield'
        when t2.survey_response_id = 1658804 then 'Senate- Lean Greenfield'
        when t2.survey_response_id = 1658805 then 'Senate- Undecided'
        when t2.survey_response_id = 1658807 then 'Senate- Lean Ernst'
        when t2.survey_response_id = 1658810 then 'Senate- Strong Ernst'
        when t2.survey_response_id = 1658813 then 'Senate- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from my_state.person c
join
((select distinct(myv_van_id)
 from my_state.person
 where ethnicity_modeled = 'H')
 union
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_activist_codes
 where activist_code_id = 4752784)) t1 on c.myv_van_id = t1.myv_van_id
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405716 --***IA US Senate ID
	and (t2.survey_response_id = 1658801	-- Strong Greenfiel
         or t2.survey_response_id = 1658804	-- Lean Greenfield
         or t2.survey_response_id = 1658805	-- Undecided
         or t2.survey_response_id = 1658807	-- Lean Ernst
         or t2.survey_response_id = 1658810	-- Strong Ernst
         or t2.survey_response_id = 1658813)	-- Support Other
 	and t2.sq_most_recent_response = 1
group by 1, 2)

union all

/* Dem ID */
(select 
 	c.county_name,
	case 
    	when t2.survey_response_id = 1668845 then 'Party- Strong Democrat'
        when t2.survey_response_id = 1668846 then 'Party- Lean Democrat'
        when t2.survey_response_id = 1668847 then 'Party- Undecided'
        when t2.survey_response_id = 1668848 then 'Party- Lean Republican'
        when t2.survey_response_id = 1668849 then 'Party- Strong Republican'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from my_state.person c
join
((select distinct(myv_van_id)
 from my_state.person
 where ethnicity_modeled = 'H')
 union
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_activist_codes
 where activist_code_id = 4752784)) t1 on c.myv_van_id = t1.myv_van_id
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 --***IA Dem ID
	and (survey_response_id = 1668845	-- Strong Democrat
         or survey_response_id = 1668846 -- Lean Democrat
         or survey_response_id = 1668847 -- Undecided
         or survey_response_id = 1668848 -- Lean Republican
         or survey_response_id = 1668849) -- Strong Republican
 	and t2.sq_most_recent_response = 1
group by 1, 2)

union all

/* Ballot Chase SQ */

(select
 	c.county_name,
	case 
    	when t2.survey_response_id = 1736473 then 'Y: Already Mailed'
        when t2.survey_response_id = 1744703 then 'Y: Ballot Collected'
        when t2.survey_response_id = 1736481 then 'Y: Commit to Return'
        when t2.survey_response_id = 1736483 then 'N: Did not recieve'
        when t2.survey_response_id = 1736937 then 'N: Voting in Person'
    end as metric,
    count(t1.myv_van_id) as metric_count
from my_state.person c
join
((select distinct(myv_van_id)
 from my_state.person
 where ethnicity_modeled = 'H')
 union
(select distinct(myv_van_id)
 from my_state_van.coord20_myv_001_activist_codes
 where activist_code_id = 4752784)) t1 on c.myv_van_id = t1.myv_van_id
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 425204  -- IA Ballot Chase
	and (t2.survey_response_id = 1736473 -- Y: Already Mailed
         or t2.survey_response_id = 1744703 -- Y: Ballot Collected
         or t2.survey_response_id = 1736481	-- Y: Commit to Return
         or t2.survey_response_id = 1756210	-- Y: Ballot PickUp Req
         or t2.survey_response_id = 1748753	-- N: Not Voting
         or t2.survey_response_id = 1736483 -- N: Did not recieve
         or t2.survey_response_id = 1736937) -- N: Voting in Person
   and t2.sq_most_recent_response = 1
group by 1, 2
);

-- STEP 16
-- DVC Deep Dive DVC Addition
/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.current_universe_reporting t1)

union all

/* Total Canvassed */

(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.current_universe_reporting t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))
 
union all

/* VBM Commits */
(select 'VBM Commits' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.current_universe_reporting t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 406163 -- IA VBM PLAN SQ
 	and sq_most_recent_response = 1
	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
           or survey_response_id = 1660742 -- Y: Commit to Request 
           or survey_response_id = 1660745 -- Y: Commit to Return 
           or survey_response_id = 1660747 -- Y: Already Returned
           or survey_response_id = 1660748 -- Y: ABR Collected
          )) 

union all 

/* Dem IDs */

(select 'Dem IDs' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.current_universe_reporting t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 -- ***IA Dem
  and sq_most_recent_response = 1
	and (survey_response_id = 1668845	-- 1 - Strong Democrat
         or survey_response_id = 1668846));	-- 2 - Lean Democrat
         
-- STEP 17

-- DVC Deep Dive DVC Addition
/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.current_universe_reporting t1)

union all

/* Total Canvassed */

(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.current_universe_reporting t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))
 
union all

/* VBM Commits */
(select 'VBM Commits' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.current_universe_reporting t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 406163 -- IA VBM PLAN SQ
 	and sq_most_recent_response = 1
	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
           or survey_response_id = 1660742 -- Y: Commit to Request 
           or survey_response_id = 1660745 -- Y: Commit to Return 
           or survey_response_id = 1660747 -- Y: Already Returned
           or survey_response_id = 1660748 -- Y: ABR Collected
          )) 

union all 

/* Dem IDs */

(select 'Dem IDs' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.current_universe_reporting t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 -- ***IA Dem
  and sq_most_recent_response = 1
	and (survey_response_id = 1668845	-- 1 - Strong Democrat
         or survey_response_id = 1668846));	-- 2 - Lean Democrat
         
-- STEP 18

-- DVC Deep Dive EVIP Tier Roll Up

select
	a.tier,
    a.total_targets,
    b.total_attempts,
    c.people_attempted,
    d.active_targets,
    (d.active_targets - e.active_people_attempted) as active_unattempted_ppl,
    f.total_canvassed,
    g.call_attempts,
    h.text_attempts,
    i.calls_canvassed,
    j.texts_canvassed,
    k.ids_collected,
    l.dem_ids_collected,
    m.vbm_status_count,
    n.vbm_commits,
    o.ppl_call_attempted,
    p.ppl_text_attempted
from
/*Total Targets */
(select t1.tier,
 	count(distinct(t1.myv_van_id)) as total_targets
 from states_ia_universes.evip_all_tiers t1
group by 1) a
left join
/*Total Attempts*/
(select t1.tier,
 	count(t2.myv_van_id) as total_attempts
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where (t2.attempt_phone = 1
         or (t2.attempt_text = 1  and t2.contact_type_name = 'SMS Text'))
 group by 1) b on a.tier = b.tier
  left join
 /* Total People Attempted */
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as people_attempted
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where (t2.attempt_phone = 1
         or (t2.attempt_text = 1  and t2.contact_type_name = 'SMS Text'))
 group by 1) c on a.tier = c.tier
 left join
 /* Active Universe */
 (select t1.tier,
 	count(distinct(t1.myv_van_id)) as active_targets
 from states_ia_universes.evip_active_tiers t1
group by 1) d on a.tier = d.tier
left join
/* # of Attempted Active Targets */
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as active_people_attempted
 from states_ia_universes.evip_active_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where (t2.attempt_phone = 1
         or (t2.attempt_text = 1  and t2.contact_type_name = 'SMS Text'))
 group by 1) e on a.tier = e.tier
 left join
 /* Total People Canvassed */
  (select t1.tier,
 	count(distinct(t2.myv_van_id)) as total_canvassed
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where (t2.phone_canvassed = 1
     or (t2.text_canvassed = 1 and t2.contact_type_name = 'SMS Text'))
 group by 1) f on a.tier = f.tier
 left join
 /* Total Call Attempts */
(select t1.tier,
 	count(t2.myv_van_id) as call_attempts
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where t2.attempt_phone = 1
 group by 1) g on a.tier = g.tier
left join
 /* Total Text Attempts */
 (select t1.tier,
 	count(t2.myv_van_id) as text_attempts
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where t2.attempt_text = 1
  and t2.contact_type_name = 'SMS Text'
 group by 1) h on a.tier = h.tier
left join 
/* Calls Canvassed */
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as calls_canvassed
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where (t2.phone_canvassed = 1)
 group by 1) i on a.tier = i.tier
left join
/* Texts Canvassed */
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as texts_canvassed
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where (t2.text_canvassed = 1)
    and t2.contact_type_name = 'SMS Text'
 group by 1) j on a.tier = j.tier
left join
/* IDs Collected */ 
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as ids_collected
  from states_ia_universes.evip_all_tiers t1
  left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
  	where survey_question_id = 408235 -- ***IA Dem
          and sq_most_recent_response = 1
 group by 1) k on a.tier = k.tier
left join
 /* Dem IDs Collected */
  (select t1.tier,
 	count(distinct(t2.myv_van_id)) as dem_ids_collected
  from states_ia_universes.evip_all_tiers t1
  left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
  	where survey_question_id = 408235 -- ***IA Dem
   		and (survey_response_id = 1668845 -- String Democrat
             or survey_response_id = 1668846) --Lean Democrat'
        and sq_most_recent_response = 1
 group by 1) l on a.tier = l.tier
 left join 
 /* VBM Status Count */
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as vbm_status_count
  from states_ia_universes.evip_all_tiers t1
  left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
  	where survey_question_id = 406163 -- VBM SQ
          and sq_most_recent_response = 1
 group by 1) m on a.tier = m.tier
 left join
/* VBM Commit Count */
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as vbm_commits
  from states_ia_universes.evip_all_tiers t1
  left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
  	where survey_question_id = 406163 -- VBM SQ
    	and (survey_response_id = 1660747 --'Y: Already Returned'
        	 or survey_response_id = 1660740 --'Y: Commit Send ABR'
             or survey_response_id = 1660742 -- Y: Commit to Request'
        	 or survey_response_id = 1660745 -- 'Y: Commit to Return'	
       		 or survey_response_id = 1660748) -- 'Y: ABR Collected'
          and sq_most_recent_response = 1
 group by 1) n on a.tier = n.tier
left join
 /* Total People Call Attempted */
(select t1.tier,
 	count(distinct(t2.myv_van_id)) as ppl_call_attempted
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where t2.attempt_phone = 1
 group by 1) o on a.tier = o.tier
 left join
  /* Total People Text Attempted */
 (select t1.tier,
 	count(distinct(t2.myv_van_id)) as ppl_text_attempted
 from states_ia_universes.evip_all_tiers t1
 left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
 where t2.attempt_text = 1
   and t2.contact_type_name = 'SMS Text'
 group by 1) p on a.tier = p.tier

order by tier;

-- STEP 19

-- DVC Addition County Roll Up After 10/10/20

/* DVC Targets County */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.region_name,
	t2.county_name,
	'Total DVC Targets' as metric,
	count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.turf t3 on t2.van_precinct_id = t3.van_precinct_id
group by 1, 2, 3
order by 2, 3)

union all

/* Total Attempts by County*/

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.region_name,
 	t2.county_name,
 	'Total Attempts' as metric,
    count(t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.attempt_phone = 1
         or t3.attempt_text = 1)
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3, 4
order by 1, 2, 3)

union all

/* Total People Attempted by County*/

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.region_name,
 	t2.county_name,
 	'Total People Attempted' as metric,
    count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.attempt_phone = 1
         or t3.attempt_text = 1)
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3, 4
order by 1, 2, 3)

union all

/* Total Canvassed by County*/

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.region_name,
 	t2.county_name,
 	'Total Canvassed' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.phone_canvassed = 1
         or t3.text_canvassed = 1)
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3, 4
order by 2, 3)

union all

/* Total VBM Commits by County*/

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.region_name,
 	t2.county_name,
 	'VBM Commits' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ
 	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
           or survey_response_id = 1660742 -- Y: Commit to Request 
           or survey_response_id = 1660745 -- Y: Commit to Return 
           or survey_response_id = 1660747 -- Y: Already Returned
           or survey_response_id = 1660748 -- Y: ABR Collected
          )
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3, 4
order by 2, 3)

union all 

/* Total VBM Status Count by County*/

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.region_name,
 	t2.county_name,
 	'VBM Status Count' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ 
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3, 4
order by 2, 3)

union all 

/* DEM IDS */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.region_name,
 	t2.county_name,
 	'DEM IDs Collected' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
  	where survey_question_id = 408235 -- ***IA Dem
   		and (survey_response_id = 1668845 -- String Democrat
             or survey_response_id = 1668846) --Lean Democrat'
        and sq_most_recent_response = 1
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3, 4
order by 2, 3);

-- STEP 20

-- DVC Addition CD Roll Up After 10/10/20

/* DVC Targets by Congressional District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.us_cong_district_latest,
	'Total DVC Targets' as metric,
	count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
group by 1, 2
order by 2)

union all

/* Call Attempts by Congressional District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.us_cong_district_latest,
 	'Call Attempts' as metric,
    sum(t2.attempt_phone) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
left join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone')
	and t2.attempt_phone = 1
 and t2.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

union all

/* Text Attempts by Congressional District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.us_cong_district_latest,
 	'Text Attempts' as metric,
 	sum(t2.attempt_text)
from states_ia_universes.ballot_chase_evip t1
join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
where t2.contact_type_name = 'SMS Text'
	and t2.attempt_text = 1
 	and t2.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total Canvassed by Congressional District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.us_cong_district_latest,
 	'Total Canvassed' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.phone_canvassed = 1
         or t3.text_canvassed = 1)
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total VBM Commits by Congressional District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.us_cong_district_latest,
 	'VBM Commits' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and t3.date_canvassed >= '2020-10-10'
 	and survey_question_id = 406163 -- IA VBM PLAN SQ
 	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
           or survey_response_id = 1660742 -- Y: Commit to Request 
           or survey_response_id = 1660745 -- Y: Commit to Return 
           or survey_response_id = 1660747 -- Y: Already Returned
           or survey_response_id = 1660748 -- Y: ABR Collected
          )
group by 1, 2, 3
order by 2)

union all

/* Total Number of IDs Received by Congressional District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.us_cong_district_latest,
	'Number of IDs' as metric,
	count(t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235
 	and t3.sq_most_recent_response = 1
  and t3.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

union all

/* Total Number of Dem IDs Received by House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.us_cong_district_latest,
	'Number of Dem IDs' as metric,
	count(t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
 	and (t3.survey_response_id = 1668845	-- 1 - Strong Democrat
         or t3.survey_response_id = 1668846)	-- 2 - Lean Democrat
 	and t3.sq_most_recent_response = 1
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

/* Total People Attempted by Congressional District */

union all

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.us_cong_district_latest,
 	'Total People Attempted' as metric,
    count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.attempt_phone = 1
         or t3.attempt_text = 1)
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3);

-- STEP 21

-- DVC Addition State House Roll Up After 10/10/20

/* DVC Targets by House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.state_house_district_latest,
	'Total DVC Targets' as metric,
	count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
group by 1, 2
order by 2)

union all

/* Call Attempts by State House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.state_house_district_latest,
 	'Call Attempts' as metric,
    sum(t2.attempt_phone) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
left join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone')
	and t2.attempt_phone = 1
 	 and t2.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

union all

/* Text Attempts by State House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.state_house_district_latest,
 	'Text Attempts' as metric,
 	sum(t2.attempt_text)
from states_ia_universes.ballot_chase_evip t1
join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
where t2.contact_type_name = 'SMS Text'
	and t2.attempt_text = 1
  and t2.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total Canvassed by State House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.state_house_district_latest,
 	'Total Canvassed' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.phone_canvassed = 1
         or t3.text_canvassed = 1)
  and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total VBM Commits by State House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.state_house_district_latest,
 	'VBM Commits' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ
 	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
           or survey_response_id = 1660742 -- Y: Commit to Request 
           or survey_response_id = 1660745 -- Y: Commit to Return 
           or survey_response_id = 1660747 -- Y: Already Returned
           or survey_response_id = 1660748 -- Y: ABR Collected
          )
 	 and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total Number of IDs Received by House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.state_house_district_latest,
	'Number of IDs' as metric,
	count(t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235
 	and t3.sq_most_recent_response = 1
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

union all

/* Total Number of Dem IDs Received by House District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.state_house_district_latest,
	'Number of Dem IDs' as metric,
	count(t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
 	and (t3.survey_response_id = 1668845	-- 1 - Strong Democrat
         or t3.survey_response_id = 1668846)	-- 2 - Lean Democrat
 	and t3.sq_most_recent_response = 1
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

/* Total People Attempted by House District */

union all

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.state_house_district_latest,
 	'Total People Attempted' as metric,
    count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.attempt_phone = 1
         or t3.attempt_text = 1)
 and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3);

-- STEP 22

-- DVC Addition State Senate Roll Up After 10/10/20

/* DVC Targets by State Senate District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.state_senate_district_latest,
	'Total DVC Targets' as metric,
	count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
group by 1, 2
order by 2)

union all

/* Call Attempts by State Senate District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.state_senate_district_latest,
 	'Call Attempts' as metric,
    sum(t2.attempt_phone) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
left join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone')
 	and t2.date_canvassed >= '2020-10-10'
	and t2.attempt_phone = 1
group by 1, 2
order by 2)

union all

/* Text Attempts by State Senate District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t3.state_senate_district_latest,
 	'Text Attempts' as metric,
 	sum(t2.attempt_text)
from states_ia_universes.ballot_chase_evip t1
join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
join my_state.person t3 on t1.myv_van_id = t3.myv_van_id
where t2.contact_type_name = 'SMS Text'
	and t2.attempt_text = 1
 and t2.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total Canvassed by State Senate District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.state_senate_district_latest,
 	'Total Canvassed' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.phone_canvassed = 1
         or t3.text_canvassed = 1)
 and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total VBM Commits by State Senate District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.state_senate_district_latest,
 	'VBM Commits' as metric,
    count(distinct t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where sq_most_recent_response = 1
 	and survey_question_id = 406163 -- IA VBM PLAN SQ
 	and (survey_response_id = 1660740 -- Y: Commit Send ABR 
           or survey_response_id = 1660742 -- Y: Commit to Request 
           or survey_response_id = 1660745 -- Y: Commit to Return 
           or survey_response_id = 1660747 -- Y: Already Returned
           or survey_response_id = 1660748 -- Y: ABR Collected
          )
 and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3
order by 2)

union all

/* Total Number of IDs Received by State Senate District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.state_senate_district_latest,
	'Number of IDs' as metric,
	count(t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235
 	and t3.sq_most_recent_response = 1
 and t3.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

union all

/* Total Number of Dem IDs Received by State Senate District */

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
	t2.state_senate_district_latest,
	'Number of Dem IDs' as metric,
	count(t1.myv_van_id) as metric_count
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
 	and (t3.survey_response_id = 1668845	-- 1 - Strong Democrat
         or t3.survey_response_id = 1668846)	-- 2 - Lean Democrat
 	and t3.sq_most_recent_response = 1
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2
order by 2)

/* Total People Attempted by State Senate District */

union all

(select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
 	t2.state_senate_district_latest,
 	'Total People Attempted' as metric,
    count(distinct(t1.myv_van_id)) as metric_count
from states_ia_universes.ballot_chase_evip t1
join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.attempt_phone = 1
         or t3.attempt_text = 1)
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2, 3);

-- STEP 23 

-- DVC Addition Weekly 10/10/20

select 'DVC Report Data Last Updated: '||to_char(convert_timezone('America/Chicago',getdate()),'Mon DD, YYYY HH12:MI am') as report_time,
    attempts.county_name,
    attempts.reporting_week_start_date,
    attempts.total_attempts,
    ids.total_ids,
    (select count(distinct(myv_van_id))
      from states_ia_universes.ballot_chase_evip) as total_universe
from
/* Total Attempts*/
(select
 	t2.county_name,
  	t3.reporting_week_start_date,
    count(t1.myv_van_id) as total_attempts
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_contacts t3 on t1.myv_van_id = t3.myv_van_id
where (t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Distributed Call'
       or t3.contact_type_name = 'Phone'
       or t3.contact_type_name = 'SMS Text')
	and (t3.attempt_phone = 1
         or t3.attempt_text = 1)
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2) attempts 
full join
/* Total IDs Collected*/
(select
 	t2.county_name,
 	t3.reporting_week_start_date,
 	count(distinct(t3.myv_van_id)) as total_ids
from states_ia_universes.ballot_chase_evip t1
left join my_state.person t2 on t1.myv_van_id = t2.myv_van_id
left join my_state_van.coord20_myv_001_responses t3 on t1.myv_van_id = t3.myv_van_id
where t3.survey_question_id = 408235 -- ***IA Dem
	and t3.sq_most_recent_response = 1
 	and t3.date_canvassed >= '2020-10-10'
group by 1, 2) ids on attempts.county_name = ids.county_name and attempts.reporting_week_start_date = ids.reporting_week_start_date;

-- STEP 24

-- DVC Deep Dive Ballot Collection Roll Up

/* Total Targets */
(select 'total_targets' as metric,
	count(myv_van_id) as metric_count
from states_ia_universes.ballot_collection_base t1)

union all

/* Total Attempts */
(select 'total_attempts' as metric,
	count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total People Attempted */
(select 'ppl_attempted' as metric,
	count(distinct (t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.attempt_phone = 1
         or t2.attempt_text = 1))

union all

/* Total Canvassed */
(select 'total_canvassed' as metric,
	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where (t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Distributed Call'
       or t2.contact_type_name = 'Phone'
       or t2.contact_type_name = 'SMS Text')
	and (t2.phone_canvassed = 1
         or t2.text_canvassed = 1))

union all

/* Biden ID */

(select  
	case 
    	when t2.survey_response_id = 1658786 then 'POTUS- Strong Biden'
        when t2.survey_response_id = 1658787 then 'POTUS- Lean Biden'
        when t2.survey_response_id = 1658788 then 'POTUS- Undecided'
        when t2.survey_response_id = 1658789 then 'POTUS- Lean Trump'
        when t2.survey_response_id = 1658790 then 'POTUS- Strong Trump'
        when t2.survey_response_id = 1658791 then 'POTUS- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405714 --***IA POTUS ID
 	and t2.sq_most_recent_response = 1
	and (t2.survey_response_id = 1658786	-- Strong Biden
         or t2.survey_response_id = 1658787	-- Lean Biden
         or t2.survey_response_id = 1658788	-- Undecided
         or t2.survey_response_id = 1658789	-- Lean Trump
         or t2.survey_response_id = 1658790	-- Strong Trump
         or t2.survey_response_id = 1658791)	-- Support Other
group by 1)

union all

/* Greenfield ID */
(select 
	case 
    	when t2.survey_response_id = 1658801 then 'Senate- Strong Greenfield'
        when t2.survey_response_id = 1658804 then 'Senate- Lean Greenfield'
        when t2.survey_response_id = 1658805 then 'Senate- Undecided'
        when t2.survey_response_id = 1658807 then 'Senate- Lean Ernst'
        when t2.survey_response_id = 1658810 then 'Senate- Strong Ernst'
        when t2.survey_response_id = 1658813 then 'Senate- Support Other'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 405716 --***IA US Senate ID
 and t2.sq_most_recent_response = 1
	and (t2.survey_response_id = 1658801	-- Strong Greenfiel
         or t2.survey_response_id = 1658804	-- Lean Greenfield
         or t2.survey_response_id = 1658805	-- Undecided
         or t2.survey_response_id = 1658807	-- Lean Ernst
         or t2.survey_response_id = 1658810	-- Strong Ernst
         or t2.survey_response_id = 1658813)	-- Support Other
group by 1)

union all

/* Dem ID */
(select 
	case 
    	when t2.survey_response_id = 1668845 then 'Party- Strong Democrat'
        when t2.survey_response_id = 1668846 then 'Party- Lean Democrat'
        when t2.survey_response_id = 1668847 then 'Party- Undecided'
        when t2.survey_response_id = 1668848 then 'Party- Lean Republican'
        when t2.survey_response_id = 1668849 then 'Party- Strong Republican'
    end as metric,    
    count(t2.myv_van_id) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where survey_question_id = 408235 --***IA Dem ID
 	and sq_most_recent_response = 1
	and (survey_response_id = 1668845	-- Strong Democrat
         or survey_response_id = 1668846 -- Lean Democrat
         or survey_response_id = 1668847 -- Undecided
         or survey_response_id = 1668848 -- Lean Republican
         or survey_response_id = 1668849) -- Strong Republican
group by 1)

union all

/* Ballot Chase SQ */

(select 
	case 
    	when t2.survey_response_id = 1736473 then 'Y: Already Mailed'
        when t2.survey_response_id = 1744703 then 'Y: Ballot Collected'
        when t2.survey_response_id = 1736481 then 'Y: Commit to Return'
 		when t2.survey_response_id = 1756210 then 'Y: Ballot PickUp Req'
 		when t2.survey_response_id = 1748753 then 'N: Not Voting'
        when t2.survey_response_id = 1736483 then 'N: Did not recieve'
        when t2.survey_response_id = 1736937 then 'N: Voting in Person'
    end as metric,
    count(t1.myv_van_id)
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_responses t2 on t1.myv_van_id = t2.myv_van_id
where t2.survey_question_id = 425204  -- IA Ballot Chase
	and (t2.survey_response_id = 1736473 -- Y: Already Mailed
         or t2.survey_response_id = 1744703 -- Y: Ballot Collected
         or t2.survey_response_id = 1736481	-- Y: Commit to Return
         or t2.survey_response_id = 1756210	-- Y: Ballot PickUp Req
         or t2.survey_response_id = 1748753	-- N: Not Voting
         or t2.survey_response_id = 1736483 -- N: Did not recieve
         or t2.survey_response_id = 1736937) -- N: Voting in Person
   and t2.sq_most_recent_response = 1
group by 1
order by 1)

union all

/* Phone Attempts */

(select
 	'Phone Attempts' as metric,
 	sum(t2.attempt_phone) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.attempt_phone = '1') 

union all

/* Text Attempts */

(select
 	'Text Attempts' as metric,
 	sum(t2.attempt_text) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.attempt_text = '1' and t2.contact_type_name = 'SMS Text') 

union all

/* Calls Canvassed */

(select
 	'Calls Canvassed' as metric,
 	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.phone_canvassed = 1) 

union all

/* Texts Canvassed */

(select
 	'Texts Canvassed' as metric,
 	count(distinct(t2.myv_van_id)) as metric_count
from states_ia_universes.ballot_collection_base t1
left join my_state_van.coord20_myv_001_contacts t2 on t1.myv_van_id = t2.myv_van_id
where t2.text_canvassed = '1'
  and t2.contact_type_name = 'SMS Text');